In [1]:
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

from encoder import *
from AttnDecoder import * 
from seq2seq import *

from build_dataset import *

/ext3/conda/1011_project/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load data 
train_file_path = {
    'source': f"data/processed/src-train.txt",
    'target': f"data/processed/tgt-train.txt"
}

test_file_path = {
    'source': f"data/processed/src-test.txt",
    'target': f"data/processed/tgt-test.txt"
}

train_dataset = QAPair(train_file_path)
test_dataset = QAPair(test_file_path)

train_dataloader = DataLoader(
            train_dataset, 
            batch_size=128, 
            shuffle=True, 
            collate_fn=partial(pad_collate_fn, pad_token=train_dataset.pad_idx)
        )

test_dataloader = DataLoader(
            test_dataset, 
            batch_size=128, 
            shuffle=True, 
            collate_fn=partial(pad_collate_fn, pad_token=test_dataset.pad_idx)
        )

In [3]:
pretrained_vectors = {}
pretrained_vectors['enc'] = torch.load('embeddings/encoder_emb.pt').float()
pretrained_vectors['dec'] = torch.load('embeddings/decoder_emb.pt').float()

# input_size = len(train_dataset.answer_vocab)
embed_size = 50  
output_size = len(train_dataset.question_vocab) 


In [4]:
def train_step(batch, model, optimizer, criterion, device):
    input = batch.input_vecs.to(device) 
    inputs_len = batch.input_lens.to(device)
    target = batch.target_vecs.to(device)
    target_len = batch.target_lens.to(device) 

    target_max_len = target.shape[-1]

    print(f'target.shape {target.shape} ')
    print(f'target_max_len {target_max_len}' )
    model.enc.train()
    model.dec.train()

    s2s_output, s2s_hidden = model(input, target, inputs_len, target_len)

    # scores = s2s_output.view(-1, s2s_output.size(-1))
    
    # optimizer.zero_grad()
    # loss = criterion(scores, target.view(-1))
    # loss.backward()
    # optimizer.step()

    # return loss.item()
    return
    

In [5]:
from tqdm import tqdm 
# TRAIN LOOP 

seq2seq = Seq2Seq(pretrained_vectors, hidden_size=600, output_size=output_size)
optim = torch.optim.SGD(seq2seq.parameters(), lr=0.1)
criterion = nn.NLLLoss()
device = "cuda" if torch.cuda.is_available() else "cpu"

seq2seq.to(device) 

for epoch in tqdm(range(1)):

    for i, data in tqdm(enumerate(train_dataloader)):
        train_step(data, seq2seq, optim, criterion, device)
        # curr_loss = train_step(data, seq2seq, optim, criterion, device)
        # print(f'epoch {epoch+1}, loss = {curr_loss}')


  0%|          | 0/1 [00:00<?, ?it/s]

target.shape torch.Size([128, 31]) 
target_max_len 31
===ENCODER FORWARD===
input shape torch.Size([128, 134])
embedded shape torch.Size([128, 134, 50])
output shape torch.Size([128, 134, 1200])
hidden shape torch.Size([4, 128, 600])
enc_out_repr torch.Size([2, 128, 600])
===DECODER FORWARD===
input shape torch.Size([128, 31])
embedded shape torch.Size([128, 31, 50])
hidden_init torch.Size([2, 128, 600])
cell_init torch.Size([2, 128, 600])
output shape torch.Size([128, 31, 600])
h_out shape torch.Size([2, 128, 600])
before calling attention
ATTENTION FORWARD
hidden shape torch.Size([128, 600])
encoder_outs shape torch.Size([128, 134, 1200])
attention module forward x unsqueeze torch.Size([128, 600, 1])
attention module forward encoder_outs torch.Size([128, 134, 1200])


0it [00:00, ?it/s]
  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [128, 1200] but got: [128, 600].